## Table of Contents
1. [Introduction](#introduction)
2. [Data loading and cleaning](#data-loading-and-cleaning)
3. [Univariate analysis](#univariate-analysis)
4. [Bivariate analysis](#bivariate-analysis)
5. [A first look at time series](#a-first-look-at-time-series)
6. [ADF tests](#adf-test)
7. [Time series decomposition](#time-series-decomposition)
8. [Auto-SARIMAX model](#auto-sarimax-model)
9. [Univariate Prophet](#univariate-prophet)
10. [Multivariate Prophet](#multivariate-prophet)
    - 10.1 [Changepoints added](#changepoints-added)
11. [Univariate Prophet revisited](#univariate-prophet-revisited)
12. [Improvements](#improvements)
    - 12.1 [Ensembling method](#ensembling-method)
    - 12.2 [Differencing the series](#differencing-the-series)
13. [Multivariate approach](#multivariate-approach)
14. [Vector Error Correction Model (VECM)](#vecm)
14. [LSTM](#lstm)
15. [Neural Prophet](#neural-prophet)
16. [Conclusion](#conclusion)


<a id="introduction" ></a>
# Introduction

In this notebook, I use several methods to predict the mean_temperature for the daily climate time series data. For beginners, I would advise to take a look on my earlier notebook:

https://www.kaggle.com/code/thuongtuandang/arima-prophet-lstm-complete-guide

Here are methods I used in this notebook:

- SARIMAX: Seasonal ARIMA + exogenous variables.
- Prophet:
    - Univariate Prophet 
    - Univariate Prophet + XG Boost on the residuals
    - Univariate Prophet on differencing time series
    - Multivariate Prophet
- VECM
- LSTM
- Neural Prophet: Neural network version of Prophet.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

<a id="data-loading-and-cleaning" ></a>
# Data Loading and Cleaning

In [ ]:
df_train = pd.read_csv('data/DailyDelhiClimateTrain.csv')
df_test = pd.read_csv('data/DailyDelhiClimateTest.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.tail()

In [ ]:
df_train.describe()

In [ ]:
df_train.isna().sum()

In [ ]:
df_test.isna().sum()

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'], format = '%Y-%m-%d')

In [ ]:
df_train['day'] = df_train['date'].dt.day
df_train['week'] = df_train['date'].dt.isocalendar().week
df_train['month'] = df_train['date'].dt.month
df_train['year'] = df_train['date'].dt.year

In [ ]:
df_train = df_train.rename(columns = {'meantemp': 'mean_temp', 'meanpressure': 'mean_pressure'})

In [ ]:
df_train

In [ ]:
def get_season(month):
    if month in [12, 1, 2]:
        return 3
    if month in [3, 4, 5]:
        return 0
    if month in [6, 7, 8]:
        return 1
    if month in [9, 10, 11]:
        return 2

df_train['season'] = df_train['month'].apply(get_season)
df_train

In [ ]:
df_train['year'].value_counts()

In [ ]:
df_train['month'].value_counts()

<a id="univariate-analysis" ></a>
# Univariate Analysis

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Histplot of the four features
columns = {'mean_temp', 'humidity', 'wind_speed', 'mean_pressure'}
fig, ax = plt.subplots(1, 4, figsize = (16,5))

for i, col in enumerate(columns):
    sns.histplot(
        data = df_train,
        x = col,
        ax=ax[i],
        kde = True,
        color = 'blue'
    )
plt.show()

Observations:

- mean_temp is bimodal: The two peaks could represent different modes for temperatures in different seasons, such as warmer temperatures in summer and cooler temperatures in winter.

- wind_speed: The wind speed distribution looks right-skewed, meaning there's a longer tail on the right side of the distribution. This could indicate that higher wind speeds are less frequent but do occur.

- humidity: The humidity distribution appears to be roughly normal or bell-shaped, indicating that most of the data falls around the central value, with fewer occurrences of extremely low or high humidity values.

- mean_pressure: The pressure distribution is quite peculiar, as it shows a very sharp peak and a narrow range of values. 

In [ ]:
# Boxplot to remove outliers (if there are any)
fig, ax = plt.subplots(1, 4, figsize = (16,5))
for i, col in enumerate(columns): 
    sns.boxplot(
        data = df_train,
        x = col,
        ax = ax[i],
        color = 'blue'
    )
plt.show()

In [ ]:
# Mean_temp over seasons and months
index = {'season', 'month'}
fig, ax = plt.subplots(1, 2, figsize = (16, 5))
for i, ind in enumerate(index):
    index_sort = df_train.groupby(ind)['mean_temp'].mean().sort_values(ascending=False).index
    sns.barplot(
        data = df_train,
        x = ind,
        y = 'mean_temp',
        ax = ax[i],
        order = index_sort,
        errorbar = None,
        estimator=np.mean
    )

The mean temperature is high on summer and low on winter.

In [ ]:
# Humidity over seasons and months
index = {'season', 'month'}
fig, ax = plt.subplots(1, 2, figsize = (16, 5))
for i, ind in enumerate(index):
    index_sort = df_train.groupby(ind)['humidity'].mean().sort_values(ascending=False).index
    sns.barplot(
        data = df_train,
        x = ind,
        y = 'humidity',
        ax = ax[i],
        order = index_sort,
        errorbar = None,
        estimator=np.mean
    )

The humidity is highest on winter, and low on spring.

In [ ]:
# Wind_speed over seasons and months
index = {'season', 'month'}
fig, ax = plt.subplots(1, 2, figsize = (16, 5))
for i, ind in enumerate(index):
    index_sort = df_train.groupby(ind)['wind_speed'].mean().sort_values(ascending=False).index
    sns.barplot(
        data = df_train,
        x = ind,
        y = 'wind_speed',
        ax = ax[i],
        order = index_sort,
        errorbar = None,
        estimator=np.mean
    )

The wind_speed is high on spring and summer, but low on autumn and winter.

In [ ]:
# mean_pressure over seasons and months
index = {'season', 'month'}
fig, ax = plt.subplots(1, 2, figsize = (16, 5))
for i, ind in enumerate(index):
    index_sort = df_train.groupby(ind)['mean_pressure'].mean().sort_values(ascending=False).index
    sns.barplot(
        data = df_train,
        x = ind,
        y = 'mean_pressure',
        ax = ax[i],
        order = index_sort,
        errorbar = None,
        estimator=np.mean
    )

It is no surprise that the mean_pressure seem not to change over the year.

<a id="bivariate-analysis" ></a>
# Bivariate Analysis

In [ ]:
# mean_temp and wind_speed
plt.figure(figsize = (14,6))
sns.scatterplot(
    data = df_train,
    x = 'mean_temp',
    y = 'wind_speed',
)
plt.title('temperature and wind speed')
plt.show()

Wind sped tends to be low when the mean_temperature is low. 

In [ ]:
# mean_temp and humidity
plt.figure(figsize = (14,6))
sns.scatterplot(
    data = df_train,
    x = 'mean_temp',
    y = 'humidity',
)
plt.title('temperature and humidity')
plt.show()

We can see that the the humidity tends to be high when the temperature is low, and it tends to be low when the temperature is high.

In [ ]:
# mean_temp and pressure
plt.figure(figsize = (14,6))
sns.scatterplot(
    data = df_train,
    x = 'mean_temp',
    y = 'mean_pressure',
)
plt.title('temperature and pressure')
plt.show()

In [ ]:
# wind_speed and humidity
plt.figure(figsize = (14,6))
sns.scatterplot(
    data = df_train,
    x = 'wind_speed',
    y = 'humidity',
)
plt.title('wind speed and humidity')
plt.show()

There may be higher humidity at lower wind speeds, which could be common in stable weather conditions without much wind.

<a id="a-first-look-at-time-series" ></a>
# A first look at time series

Because the values for mean_pressure is more or less constant, we can ignore this feature from now on.

In [ ]:
# Mean temperature over time
df_train.set_index('date', inplace=True)
df_train['mean_temp'].plot(figsize=(10, 6), title='Mean Temperature Over Time')
plt.xlabel('date')
plt.ylabel('Mean Temperature')
plt.show()

The mean_temp shows a clear seasonal pattern.

In [ ]:
# Wind speed over time
df_train['wind_speed'].plot(figsize=(10, 6), title='Wind Speed Over Time')
plt.xlabel('date')
plt.ylabel('Wind Speed')
plt.show()

The seasonal pattern for wind speed is not very clear.

In [ ]:
# humidity over time
df_train['humidity'].plot(figsize=(10, 6), title='Humidity Over Time')
plt.xlabel('date')
plt.ylabel('Humidity')
plt.show()

<a id="adf-test" ></a>
# ADF Test

In order to understand time series data, we need to see if the series are stationary. This condition is important to use ARIMA.

In [ ]:
# Because it is daily data, we can choose the rolling windown = 365
rolling_mean = df_train['mean_temp'].rolling(window=365).mean()
rolling_var = df_train['mean_temp'].rolling(window=365).std()

# Plotting the time series along with the rolling statistics
plt.figure(figsize=(14, 7))
plt.plot(df_train['mean_temp'], label='Mean Temp', color='blue')
plt.plot(rolling_mean, label='Rolling Mean', color='red', linestyle='--')
plt.plot(rolling_var, label='Rolling Variance', color='green', linestyle='--')
plt.title('Mean Temperature with Rolling Mean & Variance')
plt.xlabel('Date')
plt.ylabel('Mean Temperature')
plt.legend()
plt.show()

In [ ]:
# ADF test
from statsmodels.tsa.stattools import adfuller

result = adfuller(df_train['mean_temp'].values)
result

The ADF test results show that mean_temp is not stationary. To make it stationary, we need to difference.

In [ ]:
# Because it is daily data, we can choose the rolling windown = 365
rolling_mean = df_train['wind_speed'].rolling(window=365).mean()
rolling_var = df_train['wind_speed'].rolling(window=365).std()

# Plotting the time series along with the rolling statistics
plt.figure(figsize=(14, 7))
plt.plot(df_train['wind_speed'], label='Wind Speed', color='blue')
plt.plot(rolling_mean, label='Rolling Mean', color='red', linestyle='--')
plt.plot(rolling_var, label='Rolling Variance', color='green', linestyle='--')
plt.title('Wind Speed with Rolling Mean & Variance')
plt.xlabel('Date')
plt.ylabel('Wind Speed')
plt.legend()
plt.show()

In [ ]:
# ADF test
from statsmodels.tsa.stattools import adfuller

result = adfuller(df_train['wind_speed'].values)
result

The ADF test results show that the wind_speed is stationary (ADF statistics = -3.83 < critical value at 1% = -3.43 and p-value 0.0025 < 0.05).

In [ ]:
# Because it is daily data, we can choose the rolling windown = 365
rolling_mean = df_train['humidity'].rolling(window=365).mean()
rolling_var = df_train['humidity'].rolling(window=365).std()

# Plotting the time series along with the rolling statistics
plt.figure(figsize=(14, 7))
plt.plot(df_train['humidity'], label='Humidity', color='blue')
plt.plot(rolling_mean, label='Rolling Mean', color='red', linestyle='--')
plt.plot(rolling_var, label='Rolling Variance', color='green', linestyle='--')
plt.title('Humidity with Rolling Mean & Variance')
plt.xlabel('Date')
plt.ylabel('Humidity')
plt.legend()
plt.show()

In [ ]:
# ADF test
from statsmodels.tsa.stattools import adfuller

result = adfuller(df_train['humidity'].values)
result

The ADF test results show that humidity is stationary as well.

<a id="time-series-decomposition" ></a>
# Time series decomposition

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
# Decomposition of mean_temp

fig, ax = plt.subplots(ncols=1, nrows=4, sharex=True, figsize=(16,8))
res = seasonal_decompose(df_train['mean_temp'], period = 365, model='additive', extrapolate_trend='freq')

ax[0].set_title('Decomposition of mean_temp', fontsize=16)
res.observed.plot(ax=ax[0], legend=False, color='dodgerblue')
ax[0].set_ylabel('Observed', fontsize=14)

res.trend.plot(ax=ax[1], legend=False, color='dodgerblue')
ax[1].set_ylabel('Trend', fontsize=14)

res.seasonal.plot(ax=ax[2], legend=False, color='dodgerblue')
ax[2].set_ylabel('Seasonal', fontsize=14)

res.resid.plot(ax=ax[3], legend=False, color='dodgerblue')
ax[3].set_ylabel('Residual', fontsize=14)

plt.show()

We can see that there is seasonal pattern for mean_temp, and the increasing trend is quite smooth.

In [ ]:
# Decomposition of wind_speed
fig, ax = plt.subplots(ncols=1, nrows=4, sharex=True, figsize=(16,8))
res = seasonal_decompose(df_train['wind_speed'], period = 365, model='additive', extrapolate_trend='freq')

ax[0].set_title('Decomposition of wind speed', fontsize=16)
res.observed.plot(ax=ax[0], legend=False, color='dodgerblue')
ax[0].set_ylabel('Observed', fontsize=14)

res.trend.plot(ax=ax[1], legend=False, color='dodgerblue')
ax[1].set_ylabel('Trend', fontsize=14)

res.seasonal.plot(ax=ax[2], legend=False, color='dodgerblue')
ax[2].set_ylabel('Seasonal', fontsize=14)

res.resid.plot(ax=ax[3], legend=False, color='dodgerblue')
ax[3].set_ylabel('Residual', fontsize=14)

plt.show()

We don't see a very clear seasonal pattern for wind_speed and the trend is also quite stable.

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=4, sharex=True, figsize=(16,8))
res = seasonal_decompose(df_train['humidity'], period = 365, model='additive', extrapolate_trend='freq')

ax[0].set_title('Decomposition of humidity', fontsize=16)
res.observed.plot(ax=ax[0], legend=False, color='dodgerblue')
ax[0].set_ylabel('Observed', fontsize=14)

res.trend.plot(ax=ax[1], legend=False, color='dodgerblue')
ax[1].set_ylabel('Trend', fontsize=14)

res.seasonal.plot(ax=ax[2], legend=False, color='dodgerblue')
ax[2].set_ylabel('Seasonal', fontsize=14)

res.resid.plot(ax=ax[3], legend=False, color='dodgerblue')
ax[3].set_ylabel('Residual', fontsize=14)

plt.show()

For humidity, the seasonal pattern is not very clear, but it show there is seasonal, and we also have a decreasing trend.

<a id="auto-sarimax-model" ></a>
# Auto-SARIMAX Model 

***wind_speed and humidity are assumed to be known***

In [ ]:
df_train

In [ ]:
!pip install -qqq pmdarima

In [ ]:
import pmdarima as pm
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
train_size = int(len(df_train) * 0.8)
train, test = df_train.iloc[:train_size], df_train.iloc[train_size:]
print(train.shape)
print(test.shape)

In [ ]:
# Fitting the SARIMAX model on the training set
smodel = pm.auto_arima(train['mean_temp'], 
                    exogenous=train[['wind_speed', 'humidity']],
                    seasonal=True, 
                    m=12,   # The seasonal period, adjust if needed
                    d=None, # Let the model determine 'd'
                    D=None, # Let the model determine 'D'
                    stepwise=True,
                    trace = True,
                    error_action='ignore',  
                    suppress_warnings=True)

# Review the model summary
print(smodel.summary())

In [ ]:
smodel.plot_diagnostics(figsize=(16,8))
plt.show()

In [ ]:
train_pred, conf_int = smodel.predict(n_periods=train_size, 
                                      exogenous=train[['wind_speed', 'humidity']], 
                                      return_conf_int=True)
# Compute RMSE on the training set
train_rmse = np.sqrt(mean_squared_error(train['mean_temp'], train_pred))
print(f"Train RMSE: {train_rmse}")

# Forecast on the test set
test_pred, conf_int = smodel.predict(n_periods=len(test), 
                                     exogenous=test[['wind_speed', 'humidity']], 
                                     return_conf_int=True)

# Compute RMSE on the test set
test_rmse = np.sqrt(mean_squared_error(test['mean_temp'], test_pred))
print(f"Test RMSE: {test_rmse}")

<a id="univariate-prophet" ></a>
# Univariate Prophet

In [ ]:
df_train_reset = df_train.reset_index()
df_train_reset

In [ ]:
df_train_reset = df_train_reset.rename(columns={'date': 'ds', 'mean_temp': 'y'})
# Rename 'Date' column to 'ds' to fit Prophet's expected column name
df_train_reset = df_train_reset.rename(columns={'Date': 'ds'})
df_train_reset

In [ ]:
from prophet import Prophet

In [ ]:
from sklearn.model_selection import train_test_split
# Split the reset dataset
p_train, p_test = train_test_split(df_train_reset, test_size=0.2, shuffle=False)

# Initialize the Prophet model
model = Prophet()

# Fit the model with the DataFrame
model.fit(p_train)

In [ ]:
future = model.make_future_dataframe(periods=len(p_test))

In [ ]:
future

In [ ]:
# Predict future values
forecast = model.predict(future)

In [ ]:
# RMSE on training set
y_train_pred = forecast['yhat'][:len(p_train)].values
y_train_true = p_train['y'].values
rmse = np.sqrt(mean_squared_error(y_train_true, y_train_pred))

print(f'RMSE: {rmse}')

In [ ]:
# Plot on p_train
plt.figure(figsize=(10, 6))
plt.plot(y_train_true, label='Actual', color='blue', marker='o')
plt.plot(y_train_pred, label='Predicted', color='red', linestyle='--', marker='x')
plt.title('Actual vs. Predicted Temperatures')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()

In [ ]:
# Extract the predicted and actual values
y_pred = forecast['yhat'][-len(p_test):]  # Last 'len(p_test)' predictions
y_true = p_test['y'].values

rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print(f'RMSE: {rmse}')

In [ ]:
y_pred = y_pred.values
plt.figure(figsize=(10, 6))
plt.plot(y_true, label='Actual', color='blue', marker='o')
plt.plot(y_pred, label='Predicted', color='red', linestyle='--', marker='x')
plt.title('Actual vs. Predicted Temperatures')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()

<a id="multivariate-prophet" ></a>
# Multivariate Prophet 

***Assume wind_speed and humidity are known***

In [ ]:
model = Prophet()
model.add_regressor('wind_speed')
model.add_regressor('humidity')

In [ ]:
model.fit(p_train)

In [ ]:
future = model.make_future_dataframe(periods=len(p_test))

In [ ]:
future

In [ ]:
future['wind_speed'] = df_train_reset['wind_speed']
future['humidity'] = df_train_reset['humidity']

In [ ]:
future

In [ ]:
# Predict future values
forecast = model.predict(future)

In [ ]:
# Extract the predicted and actual values
y_pred = forecast['yhat'][-len(p_test):]  # Last 'len(p_test)' predictions
y_true = p_test['y'].values

rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print(f'RMSE: {rmse}')

In [ ]:
y_pred = y_pred.values
plt.figure(figsize=(10, 6))
plt.plot(y_true, label='Actual', color='blue', marker='o')
plt.plot(y_pred, label='Predicted', color='red', linestyle='--', marker='x')
plt.title('Actual vs. Predicted Temperatures')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()

In [ ]:
residuals = y_pred - y_true
plt.figure(figsize=(10, 6))
plt.plot(residuals, label='Residuals', color='blue')
plt.show()

In [ ]:
# Histogrm of residuals
sns.histplot(
    data = residuals,
    kde = True,
    color = 'blue'
)

The residual is a little bit left-skewed, because the increasing of mean temperature from the later half of 2015 and because this belong to test set, we do not capture well this information. Let's add change point to prophet and check the results again.

<a id="changepoints-added" ></a>
# Changepoints added

In [ ]:
model = Prophet(changepoints=['2015-05-01'])

In [ ]:
model.add_regressor('wind_speed')
model.add_regressor('humidity')

In [ ]:
model.fit(p_train)

In [ ]:
future = model.make_future_dataframe(periods=len(p_test))

In [ ]:
future

In [ ]:
future['wind_speed'] = df_train_reset['wind_speed']
future['humidity'] = df_train_reset['humidity']

In [ ]:
future

In [ ]:
# Predict future values
forecast = model.predict(future)

In [ ]:
# Extract the predicted and actual values
y_pred = forecast['yhat'][-len(p_test):]  # Last 'len(p_test)' predictions
y_true = p_test['y'].values

rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print(f'RMSE: {rmse}')

In [ ]:
y_pred = y_pred.values
plt.figure(figsize=(10, 6))
plt.plot(y_true, label='Actual', color='blue', marker='o')
plt.plot(y_pred, label='Predicted', color='red', linestyle='--', marker='x')
plt.title('Actual vs. Predicted Temperatures')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()

In [ ]:
residuals = y_pred - y_true
plt.figure(figsize=(10, 6))
plt.plot(residuals, label='Residuals', color='blue')
plt.show()

In [ ]:
# Histogrm of residuals
sns.histplot(
    data = residuals,
    kde = True,
    color = 'blue'
)

The histogram looks more symmetric with change point. We will now fit the whole data and test it on df_test. 

There are two approaches: 

- Univariate approach: we will only use the history values of mean temperature to predict it on the test set.

- Multivariate approach: for this, we would need to predict values of wind_speed and humidity first and fit it to the multivariate prophet.

<a id="univariate-prophet-revisited" ></a>
# Univariate Prophet revisited

In [ ]:
df_test

In [ ]:
df_test['date'] = pd.to_datetime(df_test['date'], format = '%Y-%m-%d')

In [ ]:
df_test = df_test.rename(columns = {'meantemp': 'mean_temp', 'meanpressure': 'mean_pressure'})

In [ ]:
df_test

In [ ]:
p_model = Prophet()

In [ ]:
p_model.fit(df_train_reset)

In [ ]:
future = p_model.make_future_dataframe(periods=len(df_test))

In [ ]:
forecast = p_model.predict(future)

In [ ]:
# RMSE on df_train
y_train_pred = forecast['yhat'][:len(df_train_reset):].values  # Last 'len(p_test)' predictions
y_train_true = df_train_reset['y'].values

rmse = np.sqrt(mean_squared_error(y_train_true, y_train_pred))

print(f'RMSE: {rmse}')

In [ ]:
# Plot on df_train
plt.figure(figsize=(10, 6))
plt.plot(y_train_true, label='Actual', color='blue', marker='o')
plt.plot(forecast['yhat'].values, label='Predicted', color='red', linestyle='--', marker='x')
plt.title('Actual vs. Predicted Temperatures')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()

In [ ]:
# Residuals plot on df_train
residuals = y_train_true - y_train_pred
plt.plot(
    residuals,
    color = 'blue'
)

In [ ]:
# Histplot on df_train
import seaborn as sns
sns.histplot(
    data=residuals,
    kde=True,
    color = 'blue'
)

It seems to have a little bit left tail, but it looks quite good.

In [ ]:
# RMSE for df_test
y_pred = forecast['yhat'][-len(df_test):]  # Last 'len(p_test)' predictions
y_true = df_test['mean_temp'].values

rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print(f'RMSE: {rmse}')

In [ ]:
# Plot for df_test
plt.figure(figsize=(10, 6))
plt.plot(y_true, label='Actual', color='blue', marker='o')
plt.plot(y_pred.values, label='Predicted', color='red', linestyle='--', marker='x')
plt.title('Actual vs. Predicted Temperatures')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()

In [ ]:
# Residuals distribution
residuals = y_true - y_pred.values
sns.histplot(
    data=residuals,
    kde=True,
    color='blue'
)

- The histogram appears to be slightly skewed to the left, which suggests that there is a tendency for the Prophet model to overestimate the values.
- The distribution of residuals is not centered around zero, indicating a bias in the forecasts.
- The tails, especially the left tail, seem to be heavier, indicating the presence of some larger errors.

In [ ]:
# Residuals plot
plt.figure(figsize=(10, 6))
plt.plot(residuals, color='blue', marker='o')
plt.xlabel('Date')
plt.ylabel('Residuals')
plt.legend()
plt.show()

From the residuals distribution and plot, we can see that the residuals is not centered around 0 and also not symmetric around the center. It could be because prophet did not capture well trends. 

In [ ]:
from prophet.plot import add_changepoints_to_plot
fig = p_model.plot(forecast)
# Add change points to the plot
a = add_changepoints_to_plot(fig.gca(), p_model, forecast)

You can change it by manually adding change point or set higher changepoint_prior_scale (default = 0.05).

<a id="improvements" ></a>
# Improvements.

<a id="ensembling-method" ></a>
# Ensembling method

We will use XG Boost on the residuals to see if the result is improved. It is good to have more features for XG Boost, and we can add cyclical features, for example: day of year, week, month and season.

In [ ]:
columns = ['ds', 'day', 'week', 'month', 'year', 'season']
df_xg = df_train_reset[columns]
df_xg

In [ ]:
df_xg['residuals'] = y_train_true - y_train_pred
df_xg

In [ ]:
df_xg['day_of_year'] = df_xg['ds'].dt.dayofyear
df_xg

In [ ]:
df_xg['day_sin'] = np.sin(2*np.pi*df_xg['day_of_year']/365)
df_xg['day_cos'] = np.cos(2*np.pi*df_xg['day_of_year']/365)

f, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))

sns.scatterplot(x=df_xg.day_sin, y=df_xg.day_cos, color='dodgerblue')
plt.show()

In [ ]:
df_xg['week_sin'] = np.sin(2*np.pi*df_xg['week']/52)
df_xg['week_cos'] = np.cos(2*np.pi*df_xg['week']/52)

f, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))

sns.scatterplot(x=df_xg.week_sin, y=df_xg.week_cos, color='dodgerblue')
plt.show()

In [ ]:
df_xg['month_sin'] = np.sin(2*np.pi*df_xg['month']/12)
df_xg['month_cos'] = np.cos(2*np.pi*df_xg['month']/12)

f, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))

sns.scatterplot(x=df_xg.month_sin, y=df_xg.month_cos, color='dodgerblue')
plt.show()

In [ ]:
df_xg['season_sin'] = np.sin(2*np.pi*df_xg['season']/4)
df_xg['season_cos'] = np.cos(2*np.pi*df_xg['season']/4)

f, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))

sns.scatterplot(x=df_xg.season_sin, y=df_xg.season_cos, color='dodgerblue')
plt.show()

In [ ]:
df_xg = df_xg.drop(columns={'ds', 'day', 'month', 'week', 'season','day_of_year'})
df_xg

In [ ]:
def create_sequences(data, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(data)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(data):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = data[i:end_ix, :], data[end_ix:out_end_ix, 1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
data = df_xg.values
n_steps_in = 365
n_steps_out = 1
X_train, y_train = create_sequences(data, n_steps_in, n_steps_out)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_train_reshaped = X_train.reshape(1097, -1) 

In [ ]:
X_train_reshaped.shape

In [ ]:
import xgboost as xgb

# Create the DMatrix data format from the numpy array, which is optimized for XGBoost.
dtrain = xgb.DMatrix(X_train_reshaped, label=y_train)

# Specify model parameters
params = {
    'max_depth': 3,  # Max depth of a tree, control over-fitting.
    'eta': 0.1,      # The learning rate.
    'objective': 'reg:squarederror',  # Regression with squared loss.
    'eval_metric': 'rmse'  # Root Mean Square Error as the evaluation metric
}

# Train the model
num_round = 100  # Number of boosting rounds
bst = xgb.train(params, dtrain, num_round)

It requires a little bit more work to construct a proper test set. We will predict the next day's mean_temp according to 365 previous days, and append this prediction to the data, and then predict the the day after based on 365 previous days (including what we have predicted).

In [ ]:
columns = []
df_test_xg = df_test[columns]
df_test_xg

In [ ]:
n = len(df_test)
df_test_xg['year'] = df_xg['year'][:n]
df_test_xg['day_sin'] = df_xg['day_sin'][:n]
df_test_xg['day_cos'] = df_xg['day_cos'][:n]
df_test_xg['week_sin'] = df_xg['week_sin'][:n]
df_test_xg['week_cos'] = df_xg['week_cos'][:n]
df_test_xg['month_sin'] = df_xg['month_sin'][:n]
df_test_xg['month_cos'] = df_xg['month_cos'][:n]
df_test_xg['season_sin'] = df_xg['season_sin'][:n]
df_test_xg['season_cos'] = df_xg['season_cos'][:n]
df_test_xg

In [ ]:
# Take the last 365 days from input_days then predict and return
def process_current(input_days, index):
    X = input_days[-365:]
    # X should be of shape (1, 3650)
    X = X.reshape(1,-1)
    dtest = xgb.DMatrix(X)
    y_pred = bst.predict(dtest)[0]
    y = np.zeros(10)
    y[0] = df_test_xg['year'][index]
    y[1] = y_pred
    y[2] = df_test_xg['day_sin'][index]
    y[3] = df_test_xg['day_cos'][index]
    y[4] = df_test_xg['week_sin'][index]
    y[5] = df_test_xg['week_cos'][index]
    y[6] = df_test_xg['month_sin'][index]
    y[7] = df_test_xg['month_cos'][index]
    y[8] = df_test_xg['season_sin'][index]
    y[9] = df_test_xg['season_cos'][index]
    return y.reshape(1,-1)

In [ ]:
X_init = X_train[-1]
X_init.shape

In [ ]:
input_days = X_init
predictions = []
for i in range(len(df_test)):
    y = process_current(input_days, i)
    # Append y[1] to the prediction
    predictions.append(y[0][1])
    # Append y to input_days and use it to predict the next day
    input_days = np.vstack((input_days, y))

In [ ]:
predictions = np.array(predictions)
predictions.shape

In [ ]:
# RMSE for df_test
# Note that y_prediction = y_pred (from Prophet) + predictions (errors from XG Boost)
y_prediction = predictions + y_pred
y_true = df_test['mean_temp'].values
rmse = np.sqrt(mean_squared_error(y_true, y_prediction))

print(f'RMSE: {rmse}')

We can see that the RMSE los decreases.

In [ ]:
# Plot for df_test
plt.figure(figsize=(10, 6))
plt.plot(y_true, label='Actual', color='blue', marker='o')
plt.plot(y_prediction.values, label='Predicted', color='red', linestyle='--', marker='x')
plt.title('Actual vs. Predicted Temperatures')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()

In [ ]:
# Residuals distribution
residuals = y_true - y_prediction
sns.histplot(
    data=residuals,
    kde=True,
    color='blue'
)

***Compared to the residuals of Prophet model alone***

- The histogram is more centered around zero, which suggests that the combined model is, on average, more accurate, with less bias in the forecasts.
- There is a noticeable improvement in the distribution of residuals, with fewer instances of large errors as compared to the Prophet-only model.
- The distribution appears to be more symmetric, indicating that the positive and negative errors are more balanced.

In [ ]:
# Residuals plot
plt.figure(figsize=(10, 6))
plt.plot(residuals, color='blue', marker='o')
plt.xlabel('Date')
plt.ylabel('Residuals')
plt.legend()
plt.show()

<a id="differencing-the-series" ></a>
# Differencing the series

Because mean_temperature is not stationary, we can try differencing it and see if the result is improved.

In [ ]:
df_train_reset

In [ ]:
columns = ['ds', 'y']
# Calculate the difference
df_diff = df_train_reset[columns]
df_diff

In [ ]:
# Calculate the difference
df_diff['y_diff'] = df_diff['y'].diff()

In [ ]:
df_diff = df_diff.dropna(subset=['y_diff'])

In [ ]:
df_diff

In [ ]:
# Check stationary for y_diff
rolling_mean = df_diff['y_diff'].rolling(window=365).mean()
rolling_var = df_diff['y_diff'].rolling(window=365).std()

# Plotting the time series along with the rolling statistics
plt.figure(figsize=(14, 7))
plt.plot(df_diff['y_diff'], label='y_diff', color='blue')
plt.plot(rolling_mean, label='Rolling Mean', color='red', linestyle='--')
plt.plot(rolling_var, label='Rolling Variance', color='green', linestyle='--')
plt.title('y_diff with Rolling Mean & Variance')
plt.xlabel('Date')
plt.ylabel('y_diff')
plt.legend()
plt.show()

In [ ]:
# ADF test
from statsmodels.tsa.stattools import adfuller

result = adfuller(df_diff['y_diff'].values)
result

The ADF test results show that mean_temp is now stationary after differencing.

In [ ]:
df_diff = df_diff.rename(columns={'y': 'mean_temp', 'y_diff': 'y'})

In [ ]:
m = Prophet()

In [ ]:
m.fit(df_diff)

In [ ]:
future = m.make_future_dataframe(periods=len(df_test)) # Adjust periods as needed
forecast = m.predict(future)

In [ ]:
forecast

In [ ]:
last_actual_y = df_train_reset['y'].iloc[-1]

# 'yhat' is forecasted 'y_diff', start the reverse transformation
forecast['yhat_cumsum'] = forecast['yhat'].cumsum()

# Add the last actual 'y' to the cumulative sum to get back to original scale
forecast['yhat_original'] = last_actual_y + forecast['yhat_cumsum']

In [ ]:
forecast

In [ ]:
# RMSE for df_test
y_pred = forecast['yhat_original'][-len(df_test):]  # Last 'len(p_test)' predictions
y_true = df_test['mean_temp'].values

rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print(f'RMSE: {rmse}')

In [ ]:
# Plot for df_test
plt.figure(figsize=(10, 6))
plt.plot(y_true, label='Actual', color='blue', marker='o')
plt.plot(y_pred.values, label='Predicted', color='red', linestyle='--', marker='x')
plt.title('Actual vs. Predicted Temperatures')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()

In [ ]:
# Residuals distribution
residuals = y_true - y_pred.values
sns.histplot(
    data=residuals,
    kde=True,
    color='blue'
)

In [ ]:
# Residuals plot
plt.figure(figsize=(10, 6))
plt.plot(residuals, color='blue', marker='o')
plt.xlabel('Date')
plt.ylabel('Residuals')
plt.legend()
plt.show()

This is even not better than univariate approach alone. 

<a id="multivariate-approach" ></a>
# Multivariate approach 

***Wind_speed and humidity are unknown***

We will first use univariate Prophet to predict wind_speed and humidity.

In [ ]:
wind_model = Prophet()

In [ ]:
df_wind = df_train_reset
df_wind = df_wind.rename(columns={'y': 'mean_temperature', 'wind_speed': 'y'})
df_wind

In [ ]:
wind_model.fit(df_wind)

In [ ]:
future = wind_model.make_future_dataframe(periods=len(df_test))

In [ ]:
wind_forecast = wind_model.predict(future)

In [ ]:
# Extract the predicted and actual values
wind_pred = wind_forecast['yhat'][-len(df_test):]  # Last 'len(p_test)' predictions
wind_true = df_test['wind_speed'].values

rmse = np.sqrt(mean_squared_error(wind_true, wind_pred))

print(f'RMSE: {rmse}')

In [ ]:
# To improve the accuracy for humidity, you may consider adding change point manually
#humidity_model = Prophet(changepoints=['2013-04-01', '2013-12-01', '2014-09-20', '2015-06-15', '2015-10-15', '2016-04-01'])

In [ ]:
# Or increase the changepoint_prior_scale
# humidity_model = Prophet(changepoint_prior_scale=0.5)

In [ ]:
humidity_model = Prophet()

In [ ]:
df_humidity = df_train_reset
df_humidity = df_humidity.rename(columns={'y': 'mean_temperature', 'humidity': 'y'})
df_humidity

In [ ]:
humidity_model.fit(df_humidity)

In [ ]:
future = humidity_model.make_future_dataframe(periods=len(df_test))

In [ ]:
humidity_forecast = humidity_model.predict(future)

In [ ]:
# Extract the predicted and actual values
humidity_pred = humidity_forecast['yhat'][-len(df_test):]  # Last 'len(p_test)' predictions
humidity_true = df_test['humidity'].values

rmse = np.sqrt(mean_squared_error(humidity_true, humidity_pred))

print(f'RMSE: {rmse}')

In [ ]:
humidity_forecast

In [ ]:
# Trend plot for humidity
from prophet.plot import add_changepoints_to_plot
fig = humidity_model.plot(humidity_forecast)
# Add change points to the plot
a = add_changepoints_to_plot(fig.gca(), humidity_model, humidity_forecast)

In [ ]:
# Humidity prediction plot
humidity_pred = humidity_pred.values
plt.figure(figsize=(10, 6))
plt.plot(humidity_true, label='Actual', color='blue', marker='o')
plt.plot(humidity_pred, label='Predicted', color='red', linestyle='--', marker='x')
plt.title('Actual vs. Predicted Humidity')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()

After predicting humidity and wind_speed, we are now ready to predict mean_temp.

In [ ]:
model = Prophet()

In [ ]:
model.add_regressor('wind_speed')
model.add_regressor('humidity')

In [ ]:
model.fit(df_train_reset)

In [ ]:
future = model.make_future_dataframe(periods=len(df_test))

In [ ]:
future['wind_speed'] = wind_forecast['yhat']
future['humidity'] = humidity_forecast['yhat']

In [ ]:
forecast = model.predict(future)

In [ ]:
# Extract the predicted and actual values
y_pred = forecast['yhat'][-len(df_test):]  # Last 'len(p_test)' predictions
y_true = df_test['mean_temp'].values

rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print(f'RMSE: {rmse}')

In [ ]:
y_pred.values

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(y_true, label='Actual', color='blue', marker='o')
plt.plot(y_pred.values, label='Predicted', color='red', linestyle='--', marker='x')
plt.title('Actual vs. Predicted Temperatures')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()


In [ ]:
# Residuals distribution
residuals = y_true - y_pred.values
sns.histplot(
    data=residuals,
    kde=True,
    color='blue'
)

In [ ]:
# Residuals plot
plt.figure(figsize=(10, 6))
plt.plot(residuals, color='blue', marker='o')
plt.xlabel('Date')
plt.ylabel('Residuals')
plt.legend()
plt.show()

Here, I think we can improve by using ensemble method for predicting humidity.

<a id="vecm" ></a>
# Vector Error Correction Model (VECM)

In [ ]:
columns = ['mean_temp', 'humidity', 'wind_speed']
df = df_train[columns]
df

As we already saw, mean_temp is not stationary while both wind_speed and humidity are stationary. The Johansen test will tell us if there is a linear combination of the three time series such that it is stationary. Note that we need this condition to use VECM.

In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
# det_order:
# -1: No deterministic term (no intercept or trend).
# 0: Includes an intercept (constant) term in the model. 
# This option models the data with a mean different from zero but without a trend.
# 1: Includes an intercept (constant) and a linear trend in the cointegration equations. 
# This implies the series have a trend but the cointegration equation corrects for it.
# 2: Includes an intercept and a quadratic trend. 
# This is less common and suggests a more complex trend structure.

# k_ar_diff: lag order

def johansen_test(data, det_order=1, k_ar_diff=1):
    result = coint_johansen(data, det_order, k_ar_diff)
    print(f'Trace Statistic: {result.lr1}')
    print(f'Critical Values (90%, 95%, 99%): {result.cvt}')
    print(f'Eigen Statistic: {result.lr2}')
    print(f'Critical Values for Eigen Statistic (90%, 95%, 99%): {result.cvm}')

johansen_test(df)


The Trace statistic tests the null hypothesis of at most r cointegrating relationships against the alternative of more than r cointegrating relationships.

- The second value (108.998041) exceeds its 99% critical value (23.1485) for the hypothesis of at most one cointegrating relationship suggesting at least two cointegrating relationships.

- The third value (12.88868224) is below the 95% critical value (18.3985) but above the 90% critical value (16.1619) for the hypothesis of at most two cointegrating relationships (r$\le$2), which might suggest some ambiguity for the presence of a third cointegrating relationship, depending on the significance level chosen.


The Maximum Eigenvalue statistic tests the null hypothesis of r cointegrating relationships against the alternative of r+1 cointegrating relationships.

- The second Eigenvalue (96.10935876) also exceeds its 99% critical value (21.7465), suggesting strong evidence for at least two cointegrating relationships.

- The third Eigenvalue (12.88868224) is above the 95% critical value (6.6349) for the hypothesis of at most two cointegrating relationships (r≤2), indicating evidence for a third cointegrating relationship.



In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
johansen_test = coint_johansen(df, det_order=-1, k_ar_diff=1)
cointegrating_vectors = johansen_test.evec

# Print the cointegrating vectors
print("Cointegrating vectors:")
print(cointegrating_vectors)

It means that 0.06 mean_temp + 0.002 humidity - 0.27 wind_speed is a stationary time series.

In [ ]:
df.shape

In [ ]:
from statsmodels.tsa.vector_ar.vecm import VECM

# Fit the VECM model
vecm = VECM(df, coint_rank=2, freq='D')
vecm_fit = vecm.fit()

In [ ]:
# Summary of the model
print(vecm_fit.summary())

In [ ]:
# Forecasting
# Forecase on test set with 95% confidence
steps = len(df_test)
forecast, lower, upper = vecm_fit.predict(steps=steps, alpha=0.05) 

In [ ]:
temp_pred = forecast[:, 0]
temp_true = df_test['mean_temp'].values
rmse = np.sqrt(mean_squared_error(temp_true, temp_pred))
print(rmse)

<a id="lstm" ></a>
# LSTM

In [ ]:
columns = ['mean_temp', 'humidity', 'wind_speed']
df = df_train[columns]
df

In [ ]:
df = df.reset_index()

In [ ]:
df['day_of_year'] = df['date'].dt.dayofyear

In [ ]:
df['day_of_year'].unique()

In [ ]:
df['day_sin'] = np.sin(2*np.pi*df['day_of_year']/365)
df['day_cos'] = np.cos(2*np.pi*df['day_of_year']/365)

f, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))

sns.scatterplot(x=df.day_sin, y=df.day_cos, color='dodgerblue')
plt.show()

In [ ]:
df.drop(['date','day_of_year'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
def create_sequences(data, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(data)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(data):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = data[i:end_ix, :], data[end_ix:out_end_ix, 0]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
data = df.values
n_steps_in = 365
n_steps_out = len(df_test)
X_train, y_train = create_sequences(data, n_steps_in, n_steps_out)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
# Remember to turn GPU on for LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from keras.optimizers import Adam


num_features = 5  
num_time_steps = 365
num_predictions = 114
optimizer = Adam(learning_rate=0.0003)

# Define the model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(num_time_steps, num_features)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_predictions))  
model.compile(optimizer=optimizer, loss='mse')

# Fit model
model.fit(X_train, y_train, epochs=100, validation_split=0.2)

In [ ]:
# Calculate MSE for training data
y = model.predict(X_train)
mse = mean_squared_error(y_train, y) 
rmse = sqrt(mse)

print("RMSE:", rmse)

In [ ]:
X_test = data[-365:]

In [ ]:
X_test.shape

In [ ]:
X_test = X_test.reshape(1, 365, 5)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred.shape

In [ ]:
y_test = df_test['mean_temp'].values.reshape(1,-1)
y_test.shape

In [ ]:
mse = mean_squared_error(y_test, y_pred) 
rmse = sqrt(mse)

print("RMSE:", rmse)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(y_test.reshape(-1,1), label='Actual', color='blue', marker='o')
plt.plot(y_pred.reshape(-1,1), label='Predicted', color='red', linestyle='--', marker='x')
plt.title('Actual vs. Predicted Temperatures')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()

<a id="neural-prophet" ></a>
# Neural Prophet

In [ ]:
!pip install -qqq neuralprophet

In [ ]:
columns = ['ds', 'y']
df_train_neural = df_train_reset[columns]

In [ ]:
df_train_neural

In [ ]:
from neuralprophet import NeuralProphet

# Initialize the model
m = NeuralProphet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False,
)

# Fit the model
metrics = m.fit(df_train_neural, freq="D")

# Create future dataframe
future = m.make_future_dataframe(df_train_neural, periods=len(df_test))

# Forecast
forecast = m.predict(future)

In [ ]:
forecast

In [ ]:
y_pred = forecast['yhat1'].values
y_true = df_test['mean_temp'].values

mse = mean_squared_error(y_true, y_pred)
rmse = sqrt(mse)
print(rmse)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(y_true, label='Actual', color='blue', marker='o')
plt.plot(y_pred, label='Predicted', color='red', linestyle='--', marker='x')
plt.title('Actual vs. Predicted Temperatures')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()

<a id="conclusion" ></a>
# Conclusion

The best model for the problem is Prophet, both multivariate (with unknown wind_speed and humidity) and univariate version perform well. When combining with XG boost on the residuals, the results for Prophet is improved. There is a reason for it:

- Prophet excels at capturing trend and seasonality, including handling holiday effects and changes in trend.

- XGBoost excels at modeling complex, nonlinear relationships and interactions between features. By focusing on the residuals, XGBoost effectively hones in on the patterns that Prophet may overlook.

LSTM and neural Prophet also perform well, but those models, especially LSTM may need more data to shine.